In [128]:
### Benjamin Tollison ###

from decimal import Decimal

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import sympy as sp
from IPython.display import Latex, Math, display
from sympy import (
    Eq,
    Function,
    Matrix,
    cos,
    cosh,
    exp,
    integrate,
    lambdify,
    pi,
    sin,
    sinh,
    symbols,
)
from sympy.solvers.pde import pdsolve
from sympy.solvers.solveset import linsolve


def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)
def displayVariable(variable:str,RHS):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
def displayVariableWithUnits(variable:str,RHS,units):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    latexUnit = sp.latex(symbols(units))
    display(Math(left + '=' + right + '\\;' +'\\left['+ latexUnit + '\\right]'))
def format_scientific(number:float):
    a = '%E' % number
    return a.split('E')[0].rstrip('0').rstrip('.') + 'E' + a.split('E')[1]
deg2rad = np.pi/180
rad2deg = 180/np.pi

In [129]:
# Spacials (not Globals)
AstroConstants = {
  'Sun':{'mu':Decimal(1.327124400419393*10**11),'req':696000},
  'Mercury':{'mu':Decimal(2.203178000000002*10**4),'req':2439.7,'soi':112500,'avg orbit radius':57910000},
  'Venus':{'mu':Decimal(3.248585920000000*10**5),'req':6051.8,'soi':616400,'avg orbit radius':(1.082041398*10**8)},
  'Earth':{'mu':Decimal(3.986004354360959*10**5),'req':6378.1365,'soi':916600,'avg orbit radius':(1.495978707*10**8)},
  'Moon':{'mu':Decimal(4.902800066163796*10**3),'req':1737.4},
  'Mars':{'mu':Decimal(4.282837362069909*10**4),'req':3396.19,'soi':577400,'avg orbit radius':227940000},
  'Jupiter':{'mu':Decimal(1.266865349218008*10**8),'req':71492,'soi':48223000,'avg orbit radius':778330000},
  'Saturn':{'mu':Decimal(3.793120749865224*10**7),'req':60268,'soi':54679000,'avg orbit radius':1429400000},
  'Uranus':{'mu':Decimal(5.793951322279009*10**6),'req':25559,'soi':51792000,'avg orbit radius':2870990000},
  'Neptune':{'mu':Decimal(6.835099502439672*10**6),'req':24764,'soi':86975000,'avg orbit radius':4504300000},
  'Pluto':{'mu':Decimal(8.696138177608749*10**2),'req':1195,'soi':15146000,'avg orbit radius':5913520000}
}
pd.DataFrame.from_dict(AstroConstants).to_csv('AstroConstants.csv') # A surprise tool that will help us later -Mickey Mouse

In [130]:
### Prolbem 1 ###
# part a
radius_aphelion = AstroConstants['Earth']['avg orbit radius']
radius_perihelion = AstroConstants['Venus']['avg orbit radius']
semimajor_transfer = 0.5*(radius_aphelion + radius_perihelion)
eccentricity_transfer = (radius_aphelion-radius_perihelion)/(radius_aphelion+radius_perihelion)
displayVariableWithUnits('a_t',semimajor_transfer,'km')
displayVariable('e_t',eccentricity_transfer)
mu_sun = float(AstroConstants['Sun']['mu'])
energy_transfer = -mu_sun/(2*semimajor_transfer)
velocity_1 = (2*(mu_sun/radius_aphelion + energy_transfer))**0.5
velocity_2 = (2*(mu_sun/radius_perihelion + energy_transfer))**0.5
displayVariableWithUnits('v_1',velocity_1,'\\frac{km}{s}')
displayVariableWithUnits('v_2',velocity_2,'\\frac{km}{s}')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [131]:
# part b
velocity_venus = (mu_sun/radius_perihelion)**0.5
displayVariableWithUnits('v_{venus}',velocity_venus,'\\frac{km}{s}')

<IPython.core.display.Math object>

In [132]:
# part c
radius_circular_earth = AstroConstants['Earth']['req'] + 550
mu_earth = float(AstroConstants['Earth']['mu'])
velocity_earth = (mu_sun/radius_aphelion)**0.5
velocity_excess_earth = velocity_1 - velocity_earth
displayVariableWithUnits('v_{\\infty/e}',velocity_excess_earth,'km/s')
semimajor_hyperbolic_earth = -mu_earth/velocity_excess_earth**2
eccentricity_hyperbolic_earth = 1 - radius_circular_earth/semimajor_hyperbolic_earth
turning_radians_earth = np.arccos(1/eccentricity_hyperbolic_earth)
displayVariableWithUnits('\\beta_e',turning_radians_earth*rad2deg,'deg')
radius_circular_venus = AstroConstants['Venus']['req'] + 400
mu_venus = float(AstroConstants['Venus']['mu'])
velocity_venus = (mu_sun/radius_perihelion)**0.5
velocity_excess_venus = velocity_2 - velocity_venus
displayVariableWithUnits('v_{\\infty/v}',velocity_excess_venus,'km/s')
semimajor_hyperbolic_venus = -mu_venus/velocity_excess_venus**2
eccentricity_hyperbolic_venus = 1 - radius_circular_venus/semimajor_hyperbolic_venus
turning_radians_venus = np.arccos(1/eccentricity_hyperbolic_venus)
displayVariableWithUnits('\\beta_v',turning_radians_venus*rad2deg,'deg')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [133]:
# part d
delta_v1 = (velocity_excess_earth**2 + 2*mu_earth/radius_circular_earth)**0.5 - (mu_earth/radius_circular_earth)**0.5
delta_v2 = (velocity_excess_venus**2 + 2*mu_venus/radius_circular_venus)**0.5 - (mu_venus/radius_circular_venus)**0.5
delta_v_total = delta_v1 + delta_v2
displayVariableWithUnits('\\Delta{v_1}',delta_v1,'km/s')
displayVariableWithUnits('\\Delta{v_2}',delta_v2,'km/s')
displayVariableWithUnits('\\Delta{v_{total}}',delta_v_total,'km/s')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [134]:
# part e
time_of_flight = np.pi*(semimajor_transfer**3/mu_sun)**0.5
displayVariableWithUnits('TOF',time_of_flight,'sec')
displayVariableWithUnits('TOF',time_of_flight/(60*60*24),'days')
time_period_venus = 2*np.pi*(radius_perihelion**3/mu_sun)**0.5
displayVariableWithUnits('TP_{venus}',time_period_venus,'sec')
phase_offset = (time_of_flight/time_period_venus)*rad2deg
displayVariableWithUnits('\\Delta{\\phi}',phase_offset,'deg')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [135]:
### Problem 2 ###
# part a
radius_earth = AstroConstants['Earth']['avg orbit radius']*np.array([np.cos(225*deg2rad),np.sin(225*deg2rad),0])
radius_mars = AstroConstants['Mars']['avg orbit radius']*np.array([1,0,0])
semi_parameter = 1.8e8
delta_true_anomoly = np.arccos((np.dot(radius_earth,radius_mars))/(np.linalg.norm(radius_earth)*np.linalg.norm(radius_mars)))
F = 1 - (np.linalg.norm(radius_mars)/semi_parameter)*(1-np.cos(delta_true_anomoly))
G = ((np.linalg.norm(radius_mars)*np.linalg.norm(radius_earth))/((mu_sun*semi_parameter)**0.5))*np.sin(delta_true_anomoly)
velocity_1 = (1/G)*(radius_mars - F*radius_earth)
angular_momentum_transfer = np.cross(radius_earth,velocity_1)
energy_transfer = np.cross(velocity_1,angular_momentum_transfer)/mu_sun - radius_earth/np.linalg.norm(radius_earth)
energy_transfer_magnitude = np.linalg.norm(energy_transfer)
displayVariable('e_t',energy_transfer_magnitude)
semimajor_transfer = semi_parameter/(1-energy_transfer_magnitude**2)
displayVariableWithUnits('a_t',semimajor_transfer,'km')
displayVariableWithUnits('a_t',semimajor_transfer/1.496e8,'AU')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [136]:
# part b
eccentric_anomoly = 2*np.arctan2(np.sqrt(1-energy_transfer_magnitude)*np.tan(delta_true_anomoly/2),np.sqrt(1+energy_transfer_magnitude))
mean_period = (mu_sun/semimajor_transfer**3)**0.5
displayVariable('E',eccentric_anomoly)
time_of_flight = (1/mean_period)*(eccentric_anomoly-energy_transfer_magnitude*np.sin(eccentric_anomoly))
displayVariableWithUnits('TOF',time_of_flight,'sec')
displayVariableWithUnits('TOF',time_of_flight/(60*60*24),'days')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [137]:
# part c
G_dot = 1 - (np.linalg.norm(radius_earth)/semi_parameter)*(1-np.cos(delta_true_anomoly))
F_dot = (F*G_dot - 1)/G
velocity_2 = F_dot*radius_earth + G_dot*velocity_1
velocity_mars = (mu_sun/np.linalg.norm(radius_mars))**0.5 * np.array([0,1,0])
displayVariableWithUnits('\\vec{v}_2',Matrix(velocity_2),'km/s')
displayVariableWithUnits('\\vec{v}_{mars}',Matrix(velocity_mars),'km/s')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [138]:
# part d
velocity_earth_hat = np.cross(-radius_earth/np.linalg.norm(radius_earth),np.array([0,0,1]))
velocity_earth = np.sqrt(mu_sun/np.linalg.norm(radius_earth))*velocity_earth_hat
velocity_excess_earth = velocity_1 - velocity_earth
velocity_excess_mars = velocity_2 - velocity_mars
velocity_excess_earth_scalar = np.linalg.norm(velocity_1)-np.linalg.norm(velocity_earth)
velocity_excess_mars_scalar = np.linalg.norm(velocity_2)-np.linalg.norm(velocity_mars)
radius_circular_earth = 1200 + AstroConstants['Earth']['req']
radius_circular_mars = 1200 + AstroConstants['Mars']['req']
mu_mars = float(AstroConstants['Mars']['mu'])
eccentricity_hyperbolic_earth = 1 + (velocity_excess_earth_scalar**2*radius_circular_earth)/mu_earth
eccentricity_hyperbolic_mars = 1 + (velocity_excess_mars_scalar**2*radius_circular_mars)/mu_mars
turning_radians_earth = np.arccos(1/eccentricity_hyperbolic_earth)
turning_radians_mars = np.arccos(1/eccentricity_hyperbolic_mars)
displayVariableWithUnits('\\vec{v}_{\\infty/e}',Matrix(velocity_excess_earth),'km/s')
displayVariableWithUnits('v_{\\infty/e}',np.linalg.norm(velocity_1)-np.linalg.norm(velocity_earth),'km/s')
displayVariableWithUnits('\\vec{v}_{\\infty/m}',Matrix(velocity_excess_mars),'km/s')
displayVariableWithUnits('v_{\\infty/m}',np.linalg.norm(velocity_2)-np.linalg.norm(velocity_mars),'km/s')
displayVariableWithUnits('\\beta_e',turning_radians_earth*rad2deg,'deg')
displayVariableWithUnits('\\beta_m',turning_radians_mars*rad2deg,'deg')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [139]:
# part e
delta_velocity_1 = np.sqrt(velocity_excess_earth_scalar**2 + (2*mu_earth)/radius_circular_earth) - np.sqrt(mu_earth/radius_circular_earth)
delta_velocity_2 = np.sqrt(velocity_excess_mars_scalar**2 + (2*mu_mars)/radius_circular_mars) - np.sqrt(mu_mars/radius_circular_mars)
delta_velocity_total = delta_velocity_1 + delta_velocity_2
displayVariableWithUnits('\\Delta{v_1}',delta_velocity_1,'\\frac{km}{s}')
displayVariableWithUnits('\\Delta{v_2}',delta_velocity_2,'\\frac{km}{s}')
displayVariableWithUnits('\\Delta{v}_{total}',delta_velocity_total,'\\frac{km}{s}')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>